# Acknowledgements
https://www.kaggle.com/allunia/pulmonary-dicom-preprocessing - DICOM preprocessing

https://www.kaggle.com/seraphwedd18/pe-detection-with-keras-model-creation - DICOM preprocessing

https://www.kaggle.com/redwankarimsony/rsna-str-pe-gradient-sigmoid-windowing/comments - DICOM windowing

In [1]:
import os
import pydicom
import vtk
import cv2

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import pylab as pl
import numpy as np
import pandas as pd

from scipy.ndimage import zoom
from pathlib import Path
from time import time
from random import shuffle, sample, randrange
from vtk.util import numpy_support

In [2]:
DATA_ROOT = Path("../input/rsna-str-pulmonary-embolism-detection")
TRAIN_ROOT = DATA_ROOT/"train"

MAX_IMAGES = 250000
TRAIN_STEPS = 5000
IMAGE_RESOLUTION = (256, 256)

train_csv = pd.read_csv(DATA_ROOT/"train.csv")

In [3]:
reader = vtk.vtkDICOMImageReader()
def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    _extent = reader.GetDataExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

    ConstPixelSpacing = reader.GetPixelSpacing()
    
    dcm_fields = [reader.GetRescaleSlope(), reader.GetRescaleOffset()]
    
    imageData = reader.GetOutput()
    pointData = imageData.GetPointData()
    arrayData = pointData.GetArray(0)
    ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
    ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')
    ArrayDicom = cv2.resize(ArrayDicom, IMAGE_RESOLUTION)
    return ArrayDicom, dcm_fields

In [4]:
def lung_window(img, dcm_fields):
    width = 1250
    length = -600
    window_min = length - (width/2)
    window_max = length + (width/2)
    slope, intercept = dcm_fields
    #img += np.abs(np.min(img))
    img = img * slope + intercept
    img[img < window_min] = window_min
    img[img > window_max] = window_max
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    #print(np.min(img), np.max(img))
    return img

def map_to_gradient(grey_img):
    rainbow_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    rainbow_img[:, :, 0] = np.clip(4 * grey_img - 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    rainbow_img[:, :, 1] =  np.clip(4 * grey_img * (grey_img <=0.75), 0,1) + np.clip((-4*grey_img + 4) * (grey_img > 0.75), 0, 1)
    rainbow_img[:, :, 2] = np.clip(-4 * grey_img + 2, 0, 1.0) * (grey_img > 0) * (grey_img <= 1.0)
    return rainbow_img

def rainbow_window(img, dcm_fields):
    grey_img = lung_window(img, dcm_fields)
    return map_to_gradient(grey_img)

def all_channels_window(img, dcm_fields):
    grey_img = lung_window(img, dcm_fields) * 3.0
    all_chan_img = np.zeros((grey_img.shape[0], grey_img.shape[1], 3))
    all_chan_img[:, :, 2] = np.clip(grey_img, 0.0, 1.0)
    all_chan_img[:, :, 0] = np.clip(grey_img - 1.0, 0.0, 1.0)
    all_chan_img[:, :, 1] = np.clip(grey_img - 2.0, 0.0, 1.0)
    return all_chan_img

In [5]:
studies = os.listdir(TRAIN_ROOT)
#studies = studies[:len(studies)//16]

func = lambda x: int((2**15 + x)*(255/2**16))
int16_to_uint8 = np.vectorize(func)

def load_scans(dcm_path):
    # otherwise we sort by ImagePositionPatient (z-coordinate) or by SliceLocation
    slices = []
    fields = []
    for file in os.listdir(dcm_path):
        image, dcm_fields = get_img(dcm_path + "/" + file)
        image = rainbow_window(image, dcm_fields)
        slices.append(image)
        fields.append(dcm_fields)

    return slices, fields

def filter_scanner(raw_pixelarrays):
    # in OSIC we find outside-scanner-regions with raw-values of -2000. 
    # Let's threshold between air (0) and this default (-2000) using -1000
    raw_pixelarrays[raw_pixelarrays <= -1000] = -1000
    return raw_pixelarrays

In [6]:
image_labels = train_csv[['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID', 'pe_present_on_image']]

exam_label_columns = ['StudyInstanceUID', 'negative_exam_for_pe', 'rv_lv_ratio_gte_1',
                      'rv_lv_ratio_lt_1', 'leftsided_pe', 'chronic_pe', 'rightsided_pe',
                      'acute_and_chronic_pe', 'central_pe', 'indeterminate']
exam_labels = train_csv[exam_label_columns].drop_duplicates('StudyInstanceUID')

In [7]:
negative_indices = list(image_labels.loc[image_labels['pe_present_on_image'] == 0].axes[0])
positive_indices = list(image_labels.loc[image_labels['pe_present_on_image'] == 1].axes[0])

all_indices = negative_indices + positive_indices

# ratio of selected max number of images to the total number of images
# note that this ratio could be adjusted since the entire dataset is huge
ratio = MAX_IMAGES / len(all_indices)

positive_count = len(positive_indices)#int(ratio * len(positive_indices))
negative_count = MAX_IMAGES - positive_count

positive_images = sample(positive_indices, positive_count)
negative_images = sample(negative_indices, negative_count)

training_indices = list(exam_labels.axes[0])#positive_images + negative_images
shuffle(training_indices)

# Pick a batch of data indices with a specified number of positive samples
def pick_batch(batch_size, positive_count):
    #total_positive = batch_size // positive_count
    
    indices = sample(positive_indices, positive_count) + sample(negative_indices, batch_size - positive_count)
    
    shuffle(indices)
    
    return indices

# Pick a batch of data from all images for validation
def pick_validation_batch(batch_size):
    indices = sample(all_indices, batch_size)
    
    shuffle(indices)
    
    return indices

print("Ratio:", ratio)
print("Total scans:", len(all_indices))
print("Total positive scans:", len(positive_indices))
print(f"Max images, positve_count: {MAX_IMAGES}, {positive_count}")

Ratio: 0.13961847297600685
Total scans: 1790594
Total positive scans: 96540
Max images, positve_count: 250000, 96540


In [8]:
def load_scans_from_study(study):
    scans = []
    fields = []
    series = []
    for s in os.listdir(TRAIN_ROOT/study):
        series.append(s)
        scan_set, dcm_fields = load_scans(str(TRAIN_ROOT/study/s))
        scans.append(scan_set)
        fields.append(dcm_fields)
        
    return series, scans, fields

def load_individual_scan(scan_path):
    scan, fields = get_img(scan_path)
    scan = rainbow_window(scan, fields)
    return scan

def load_batch_scans(scan_paths):
    scans = np.zeros((len(scan_paths), IMAGE_RESOLUTION[0], IMAGE_RESOLUTION[1], 3))
    for i, path in enumerate(scan_paths):
        s, f = get_img(path)
        s = rainbow_window(s, f)
        scans[i] = s
        
    return scans

def batch_images_labels_from_indices(indices):
    scan_paths = []
    labels = np.array(image_labels['pe_present_on_image'][pd.Index(indices)]).astype(np.int32)
    labels = tf.reshape(labels, (batch_size, 1))
    for index in indices:
        study = image_labels['StudyInstanceUID'][index]
        series = image_labels['SeriesInstanceUID'][index]
        scan = image_labels['SOPInstanceUID'][index]

        scan_paths.append(str(TRAIN_ROOT/('/'.join([study, series, scan+".dcm"]))))

    scans = augment(load_batch_scans(scan_paths))

    return labels, scans

In [9]:
DEFAULT_PARAMS = { 'k': 12, 'depth': 100, 'theta': 0.5, 'bottleneck': True }

class DenseLayer(layers.Layer):
    def __init__(self, k, bottleneck=False):
        super(DenseLayer, self).__init__()

        self.necked = bottleneck

        if bottleneck:
            self.bn1 = layers.BatchNormalization()
            self.relu1 = layers.ReLU()
            self.bottleneck = layers.Conv3D(4*k, 1, padding='same')

        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()
        self.conv = layers.Conv3D(k, 3, padding='same')

        if not self.necked:
            self.dropout = layers.Dropout(rate=0.2)
        self.concat = layers.Concatenate()

    def call(self, input_tensor, training):
        conv = input_tensor

        if self.necked:
            conv = self.bn1(conv, training)
            conv = self.relu1(conv)
            conv = self.bottleneck(conv)

        conv = self.bn2(conv, training)
        conv = self.relu2(conv)
        conv = self.conv(conv)

        if not self.necked:
            conv = self.dropout(conv, training)
        return self.concat([input_tensor, conv])

    def build_graph(self, input_shape):
        no_batch = input_shape[1:]
        self.build(input_shape)

        inputs = keras.Input((no_batch))
        self.call(inputs)

class Transition(layers.Layer):
    def __init__(self, input_filters, theta=0.5):
        super(Transition, self).__init__()
        self.bn = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.conv = layers.Conv3D(int(input_filters*theta), 1, padding='same')
        self.pool = layers.AveragePooling3D(2, strides=2)

    def call(self, input_tensor, training):
        conv = self.bn(input_tensor, training)
        conv = self.relu(conv)
        conv = self.conv(conv)
        return self.pool(conv)

class DenseBlock(layers.Layer):
    def __init__(self, k, depth, bottleneck):
        super(DenseBlock, self).__init__()
        self.lays = []
        for i in range(depth):
            self.lays.append(DenseLayer(k, bottleneck))

    def call(self, input_tensor, training):
        conv = input_tensor
        for layer in self.lays:
            conv = layer(conv, training)

        return conv

class DenseNet(keras.Model):
    def __init__(self, k=DEFAULT_PARAMS['k'],
                       depth=DEFAULT_PARAMS['depth'],
                       theta=DEFAULT_PARAMS['theta'],
                       bottleneck=DEFAULT_PARAMS['bottleneck']):
        super(DenseNet, self).__init__()

        self.conv_init = layers.Conv3D(2*k, 3, padding='same')

        if bottleneck:
            block_depth = ((depth-3) // 3) // 2
        else:
            block_depth = (depth-1) // 3

        self.block1 = DenseBlock(k, block_depth, bottleneck)
        self.trans1 = Transition(2*k + k*block_depth, theta=theta)
        self.block2 = DenseBlock(k, block_depth, bottleneck)
        self.trans2 = Transition((2*k + k*block_depth)/2 + k*block_depth, theta=theta)
        self.block3 = DenseBlock(k, block_depth, bottleneck)
        #self.trans3 = Transition((2*k + k*block_depth)/2 + k*block_depth, theta=theta)
        #self.block4 = DenseBlock(k, block_depth, bottleneck)
        #self.trans4 = Transition((2*k + k*block_depth)/2 + k*block_depth, theta=theta)
        #self.block5 = DenseBlock(k, block_depth, bottleneck)
        
        self.pool = layers.GlobalAveragePooling3D()
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(units=len(exam_label_columns)-1, activation='sigmoid')
        
        self.build_graph((1, 64, 64, 64, 3))
        
    def call(self, input_tensor, training=False):
        conv = self.conv_init(input_tensor)

        conv = self.block1(conv, training)
        conv = self.trans1(conv, training)
        conv = self.block2(conv, training)
        conv = self.trans2(conv, training)
        conv = self.block3(conv, training)
        #conv = self.trans3(conv, training)
        #conv = self.block4(conv, training)
        #conv = self.trans4(conv, training)
        #conv = self.block5(conv, training)

        conv = self.pool(conv)
        conv = self.flatten(conv)
        return self.dense(conv)

    def build_graph(self, input_shape):
        input_shape_nobatch = input_shape[1:]
        self.build(input_shape)#_nobatch)

        inputs = tf.keras.Input(shape=input_shape_nobatch)

        if not hasattr(self, 'call'):
            raise AttributeError("User should define 'call' method in sub-class model!")

        self.__call__(inputs, True)
        
image_model = DenseNet()
image_model.summary()

Model: "dense_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 64, 64, 64, 24)    1968      
_________________________________________________________________
dense_block (DenseBlock)     (None, 64, 64, 64, 216)   347712    
_________________________________________________________________
transition (Transition)      (None, 32, 32, 32, 108)   24300     
_________________________________________________________________
dense_block_1 (DenseBlock)   (None, 32, 32, 32, 300)   417600    
_________________________________________________________________
transition_1 (Transition)    (None, 16, 16, 16, 150)   46350     
_________________________________________________________________
dense_block_2 (DenseBlock)   (None, 16, 16, 16, 342)   452544    
_________________________________________________________________
global_average_pooling3d (Gl (None, 342)               0 

In [10]:
# Model definitions
keras.backend.clear_session()

batch_size = 1
image_opt = keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True)

loss_weights = np.full((batch_size, 1), len(training_indices)/len(positive_images))

def loss_function(labels, logits, weighted=True):
    loss = keras.losses.binary_crossentropy(labels, logits, from_logits=True, label_smoothing=0.05)
    
    if weighted:
        weights = tf.cast(tf.math.greater(labels, 0), tf.float32)*loss_weights
        weights += tf.cast(tf.math.equal(weights, 0), tf.float32)
        weights = tf.reshape(weights, (weights.shape[0],))
        
        loss = tf.math.multiply(loss, weights)
        
    print(loss)
    assert(False)

    loss = tf.math.reduce_mean(loss)
    
    return loss

l = keras.losses.BinaryCrossentropy(from_logits=True)

#@tf.function
def image_train_step(image, labels):
    with tf.GradientTape() as tape:
        logits = image_model(image)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(tf.cast(labels, tf.float32), logits)
    
    gradients = tape.gradient(loss, image_model.trainable_variables)
    image_opt.apply_gradients(zip(gradients, image_model.trainable_variables))
    
    return loss, logits

@tf.function
def image_validation_step(image, labels):
    logits = image_model(image)
    loss = loss_function(labels, logits, weighted=False)
    
    return loss, logits

def augment(images):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_flip_up_down(images)
    images = tf.image.per_image_standardization(images)
    
    return images

# Keeps track of absolute accuracy
# i.e. All labels correct => 1,
#               otherwise => 0
class MultiLabelBinaryAccuracy():
    def __init__(self):
        self.count = 0
        self.sum = 0
        
    # Expecting shapes like [label_count]
    def individual_multilabel_accuracy(self, label, logits):
        local_count = 0
        local_sum = 0
        for lab, log in zip(label, logits):
            if lab != log:
                return 0
            
        return 1
        
    # Expecting shapes like [batch_size, label_count]
    def __call__(self, labels, logits):
        batch_accuracies = []
        for i in range(labels.shape[0]):
            batch_accuracies.append(self.individual_multilabel_accuracy(labels[i], logits[i]))
            
        self.sum += tf.math.reduce_mean(batch_accuracies)
        self.count += labels.shape[0]
        
    def result(self):
        return self.sum / self.count

VALIDATION_STEPS = 500
# Run validation every VALIDATION_FREQUENCY train steps
VALIDATION_FREQUENCY = 5000

loss_met = keras.metrics.Mean()
acc_met = MultiLabelBinaryAccuracy()

total_iterations = len(training_indices)//batch_size

for i in range(TRAIN_STEPS):#range(total_iterations):
    indices = sample(training_indices, 1)
    train_study = list(train_csv['StudyInstanceUID'][indices])[0]
    scans = np.array(load_scans_from_study(train_study)[1])
    rescale_factors = (1, 64/scans.shape[1], 64/scans.shape[2], 64/scans.shape[3], 1)
    scans = zoom(scans, rescale_factors, order=0)
    labels = np.array(exam_labels[exam_label_columns[1:]].loc[indices])
    loss, logits = image_train_step(scans, labels)
    loss_met(loss)
    acc_met(labels, logits)

    print(f"Batch {i} of {TRAIN_STEPS} loss, accuracy: {loss_met.result()}, {acc_met.result()}      \r", end='')
    
image_model.save_weights("image_weights.h5")